In [96]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv("streams.csv")
df

,Stream Name,Supply,Target,Cp,U
0,Heated Methanol,43.07,250.0,24.807,1
1,Cooled Product,363.40,89.0,14.101,2
2,Feed To Column-2,156.80,139.0,10.326,1
3,Cooled Water Stream,165.30,30.0,3.420,1
4,DME Final Prouct,45.90,30.0,4.525,2
5,T-100 Condenser,45.90,45.8,20008.333,1
6,T-100 Reboiler,165.70,165.8,15538.889,1
7,T-101 Reboiler,165.30,165.4,23491.667,1
8,T-101 Condenser,126.00,125.9,22375.000,1
9,HU,184.00,183.9,35612.900,1


In [5]:
def is_hot_cold(row):
    if row["Supply"] > row["Target"]:
        return "Hot"
    else:
        return "Cold"
def get_enthalpy(row):
    return (row["Target"] - row["Supply"]) *row["Cp"]

In [6]:
df["Stream Type"] = df.apply(is_hot_cold,axis=1)
df["Delta H"] = df.apply(get_enthalpy,axis=1)
df["Abs Delta H"] = df["Delta H"].apply(abs)
assert abs(df["Delta H"].sum()) < 1, "Process is unbalanced, check that net enthalpy is close to 0"

In [7]:
df

,Stream Name,Supply,Target,Cp,U,Stream Type,Delta H,Abs Delta H
0,Heated Methanol,43.07,250.0,24.807,1,Cold,5133.31251,5133.31251
1,Cooled Product,363.40,89.0,14.101,2,Hot,-3869.31440,3869.31440
2,Feed To Column-2,156.80,139.0,10.326,1,Hot,-183.80280,183.80280
3,Cooled Water Stream,165.30,30.0,3.420,1,Hot,-462.72600,462.72600
4,DME Final Prouct,45.90,30.0,4.525,2,Hot,-71.94750,71.94750
5,T-100 Condenser,45.90,45.8,20008.333,1,Hot,-2000.83330,2000.83330
6,T-100 Reboiler,165.70,165.8,15538.889,1,Cold,1553.88890,1553.88890
7,T-101 Reboiler,165.30,165.4,23491.667,1,Cold,2349.16670,2349.16670
8,T-101 Condenser,126.00,125.9,22375.000,1,Hot,-2237.50000,2237.50000
9,HU,184.00,183.9,35612.900,1,Hot,-3561.29000,3561.29000


In [8]:
cold_df = df[df["Stream Type"] == "Cold"].copy()
hot_df = df[df["Stream Type"] == "Hot"].copy()


cold_df["Low"],cold_df["High"] = cold_df["Supply"],cold_df["Target"]
hot_df["Low"],hot_df["High"] = hot_df["Target"],hot_df["Supply"]

cold_df = cold_df.sort_values(by=["Low","High"])
hot_df = hot_df.sort_values(by=["Low","High"])



In [9]:
cold_df

,Stream Name,Supply,Target,Cp,U,Stream Type,Delta H,Abs Delta H,Low,High
10,CU,5.00,15.0,335.121,1,Cold,3351.21000,3351.21000,5.00,15.0
0,Heated Methanol,43.07,250.0,24.807,1,Cold,5133.31251,5133.31251,43.07,250.0
7,T-101 Reboiler,165.30,165.4,23491.667,1,Cold,2349.16670,2349.16670,165.30,165.4
6,T-100 Reboiler,165.70,165.8,15538.889,1,Cold,1553.88890,1553.88890,165.70,165.8


In [21]:
def inbetween(row,value):
    low = row["Low"]
    high = row["High"]
    c_p = row["Cp"]
    if value >= low and value < high:
        return c_p
    else:
        return 0
def get_regions(df):
    streams = df[["Low","High","Cp"]].copy()
    partitions = df["Low"].tolist()
    partitions.extend(df["High"].tolist())

    partitions = sorted(list(set(partitions)))
    regions = []
    for i,pt in enumerate(partitions[:-1]):
        tot_cp = streams.apply(inbetween, value=pt,axis=1).sum()
        info = [pt,partitions[i+1],tot_cp,tot_cp*(partitions[i+1]- pt)]
        regions.append(info)

    region_df = pd.DataFrame(regions,columns=["cold","hot","cp","heat_load"])
    region_df["heat_load_sum"] = region_df["heat_load"].expanding(1).sum()
    return region_df

In [28]:
hot_reg = get_regions(hot_df)
cold_reg = get_regions(cold_df)

all_loads = list(set(
    hot_reg["heat_load_sum"].tolist() + cold_reg["heat_load_sum"].tolist()
)) 
all_loads.sort()
all_loads

[125.53099999999998,
 2127.1588000000284,
 2274.5608000000284,
 2921.0857000000287,
 3351.21,
 5160.337799999901,
 5388.110799999901,
 5883.787399999901,
 6032.715899999902,
 6294.994499999902,
 6383.36961,
 8735.017009999867,
 8742.459109999867,
 9857.694599999699,
 10298.82871000022,
 12387.413999999699,
 12387.57811000022]

In [32]:
cold_reg

,cold,hot,cp,heat_load,heat_load_sum
0,5.00,15.00,335.121,3351.21000,3351.21000
1,15.00,43.07,0.000,0.00000,3351.21000
2,43.07,165.30,24.807,3032.15961,6383.36961
3,165.30,165.40,23516.474,2351.64740,8735.01701
4,165.40,165.70,24.807,7.44210,8742.45911
5,165.70,165.80,15563.696,1556.36960,10298.82871
6,165.80,250.00,24.807,2088.74940,12387.57811


In [83]:
df = hot_reg.copy()
row = 0
heat_load_sums = df["heat_load_sum"].tolist()

def final_regions(hot_reg, cold_reg):
    all_loads = hot_reg["heat_load_sum"].tolist() + cold_reg["heat_load_sum"].tolist()
    
    all_loads.sort()
    cold_temp = cold_reg.iloc[0,0]
    hot_temp = hot_reg.iloc[0,0]
    current_load = 0
    entries = []
    for target_load in all_loads[:-1]:
        if target_load in hot_reg["heat_load_sum"].tolist():
            hot_in = hot_temp  

            # If statement for duplicate loads (regions where there are no cold/hot streams)
            if current_load == target_load:
                hot_out = hot_reg[hot_reg["heat_load_sum"] == target_load]["hot"].iloc[-1]
            else:
                hot_out = hot_reg[hot_reg["heat_load_sum"] == target_load]["hot"].iloc[0]
            hot_cp = hot_reg[hot_reg["heat_load_sum"] == target_load]["cp"].iloc[-1]
            
            # Find Correct Row
            cold_in = cold_temp
            row = find_row(cold_reg, target_load)
            # print("Find Cold",row)
            cold_cp = cold_reg.iloc[row,2]
            cold_out = (target_load - current_load)/cold_cp + cold_in

        elif target_load in cold_reg["heat_load_sum"].tolist():
            cold_in = cold_temp 

            # If statement for duplicate loads (regions where there are no cold/hot streams)
            if current_load == target_load:
                cold_out = cold_reg[cold_reg["heat_load_sum"] == target_load]["hot"].iloc[-1]
            else:
                cold_out = cold_reg[cold_reg["heat_load_sum"] == target_load]["hot"].iloc[0]
            cold_cp = cold_reg[cold_reg["heat_load_sum"] == target_load]["cp"].iloc[-1]
            
            hot_in = hot_temp
            row = find_row(hot_reg, target_load)
            # print("Find Hot", row,target_load)
            hot_cp = hot_reg.iloc[row,2]
            hot_out = (target_load - current_load)/hot_cp + hot_in

        hot_temp = hot_out
        cold_temp = cold_out
        current_load = target_load
        new_entry={
            "cold_in":cold_in,"cold_out":cold_out,"cold_cp":cold_cp,
            "hot_in":hot_in,"hot_out":hot_out,"hot_cp":hot_cp,
            "heat_load":target_load
        }
        entries.append(new_entry)
    return pd.DataFrame(entries)
    

def find_row(df, load):
    ls = [0] + df["heat_load_sum"].tolist()
    for i, val in enumerate(ls[:-1]):
        if load > val and load <= ls[i+1]:
            return i
    return len(ls) -1

In [84]:
final = final_regions(hot_reg,cold_reg)
final

,cold_in,cold_out,cold_cp,hot_in,hot_out,hot_cp,heat_load
0,5.000000,5.374584,335.121,30.000000,45.800000,7.945,125.53100
1,5.374584,11.347435,335.121,45.800000,45.900000,20016.278,2127.15880
2,11.347435,11.787282,335.121,45.900000,89.000000,3.420,2274.56080
3,11.787282,13.716510,335.121,89.000000,125.900000,17.521,2921.08570
4,13.716510,15.000000,0.000,125.900000,125.919208,22392.521,3351.21000
5,15.000000,43.070000,0.000,125.919208,125.919208,22392.521,3351.21000
6,43.070000,115.998117,24.807,125.919208,126.000000,22392.521,5160.33780
7,115.998117,125.179921,24.807,126.000000,139.000000,17.521,5388.11080
8,125.179921,145.161240,24.807,139.000000,156.800000,27.847,5883.78740
9,145.161240,151.164727,24.807,156.800000,165.300000,17.521,6032.71590


In [94]:
def plot_bcc(final_df):
    # getting cold points
    cold_points = [[final_df.iloc[0,1],0]]
    cold_points = cold_points + final_df[["cold_in","heat_load"]].values.tolist() 
    cold_points = np.array(cold_points)
    plt.plot(cold_points[:,])

In [95]:
plot_bcc(final)

[[0, 5.374584105442512], [5.0, 125.53099999999998], [5.374584105442512, 2127.1588000000284], [11.347435105529133, 2274.5608000000284], [11.787282205531818, 2921.0857000000287], [13.716510454432964, 3351.21], [15.0, 3351.21], [43.07, 5160.337799999901], [115.99811706372802, 5388.110799999901], [125.17992058692712, 5883.787399999901], [145.16124037569642, 6032.715899999902], [151.16472729471124, 6294.994499999902], [161.7374930463136, 6383.36961], [165.3, 8735.017009999867], [165.4, 8742.459109999867], [165.7, 9857.694599999699], [165.77165621135234, 10298.82871000022], [165.8, 12387.413999999699]]


In [93]:
final["cold_in","heat_load"]]

SyntaxError: unmatched ']' (<ipython-input-93-97aea624e9e2>, line 1)

In [18]:
# Converting to Regions of Heat 

regions = []
delta_T = 10

In [42]:
curr_temp

5.0

In [45]:
i_low

3

In [46]:
cold_regions

[[5.0, 15.0, 335.121], [43.07, 165.3, 24.807], [165.3, 165.7, 23491.667]]